# Assignment 2 - Game 1: Quoridor

## Instructions

This is a **self-contained notebook** - everything you need is here!

### Quick Start
1. **Run all cells** up to Section 4 (this loads the game client)
2. **Implement your solver** in Section 5
3. **Update configuration** in Section 7
4. **Play the game** in Section 7

### What You Need To Do
- Focus ONLY on implementing `my_agent()` function (Section 5)
  - You can create various players, you'll be able to select your preferred one.
- Everything else is provided for you!

### About Quoridor
Quoridor is a 2-player abstract strategy game where:
- Players start at opposite ends of a 9x9 board
- Goal: Be the first to reach the opposite side
- Each turn: Either move your pawn OR place a wall to block opponent
- Each player has 10 walls to use during the game
- Walls must not completely block any player from reaching their goal

---
## Section 1: Setup

**Run this cell (no changes needed)**

In [ ]:
import requests
import json
import time
import random
from typing import List, Optional, Tuple, Any, Dict
from copy import deepcopy
from collections import deque

print("✅ Dependencies imported")

BASE_URL = 'https://ie-aireasoning-gr4r5bl6tq-ew.a.run.app'  # Your Cloud Run URL

print("✅ Configuration loaded")

---
## Section 2: Game Client Library

**Run this cell (no changes needed)**

This defines the game client that handles all server communication.

In [ ]:
class GameClient:
    def __init__(self, base_url: str, token: str, debug: bool = False):
        self.base_url = base_url.rstrip('/')
        self.token = token
        self.debug = debug

    def _make_request(self, endpoint: str, params: dict, max_retries: int = 10) -> dict:
        params['TOKEN'] = self.token
        url = f'{self.base_url}{endpoint}'

        for attempt in range(max_retries):
            try:
                if self.debug:
                    print(f"[DEBUG] Request: {endpoint}")
                    print(f"[DEBUG] Params: {params}")

                response = requests.get(url, params=params, timeout=30)

                if self.debug:
                    print(f"[DEBUG] Response [{response.status_code}]: {response.text[:200]}")

                if response.status_code == 200:
                    if response.text:
                        try:
                            return response.json()
                        except (json.JSONDecodeError, ValueError) as e:
                            if self.debug:
                                print(f"[DEBUG] Non-JSON response: {response.text[:100]}")
                            return {}
                    return {}
                else:
                    print(f"⚠️  HTTP {response.status_code}: {response.text[:200]}")

            except requests.exceptions.Timeout:
                print(f"⚠️  Request timeout (attempt {attempt + 1}/{max_retries})")
            except requests.exceptions.RequestException as e:
                print(f"⚠️  Request error: {e} (attempt {attempt + 1}/{max_retries})")
            except Exception as e:
                print(f"⚠️  Unexpected error: {type(e).__name__}: {e} (attempt {attempt + 1}/{max_retries})")

            if attempt < max_retries - 1:
                time.sleep(1)

        raise Exception(f"Failed to connect to {endpoint} after {max_retries} attempts")

    def create_match(self, game_type: str, num_games: int, multiplayer: bool = False) -> str:
        response = self._make_request('/new-match', {
            'game-type': game_type,
            'num-games': str(num_games),
            'multi-player': 'True' if multiplayer else 'False'
        })
        
        if 'match-id' not in response:
            print(f"❌ Server response missing 'match-id'. Response: {response}")
            raise KeyError(f"Server response missing 'match-id'. Got: {response}")
        
        return response['match-id']

    def join_match(self, match_id: str) -> dict:
        response = self._make_request('/join-match', {
            'match-id': match_id
        })
        return response

    def get_game_state(self, match_id: str, game_index: int) -> dict:
        return self._make_request('/game-state-in-match', {
            'match-id': match_id,
            'game-index': str(game_index)
        })

    def get_match_state(self, match_id: str) -> dict:
        return self._make_request('/match-state', {
            'match-id': match_id
        })

    def make_move(self, match_id: str, player: str, move: Any) -> bool:
        move_str = move if isinstance(move, str) else json.dumps(move)
        
        self._make_request('/make-move-in-match', {
            'match-id': match_id,
            'player': player,
            'move': move_str
        })
        return True

print("✅ GameClient loaded")

def play_game(
    solver,
    base_url: str,
    token: str,
    game_type: str,
    game_class,
    multiplayer: bool = False,
    match_id: Optional[str] = None,
    num_games: int = 1,
    debug: bool = False,
    verbose: bool = True
) -> Tuple:
    client = GameClient(base_url, token, debug=debug)
    session_turn_times: List[float] = []

    if match_id is None:
        if verbose:
            print(f"🎮 Creating new match: {num_games} x {game_type}")
        match_id = client.create_match(game_type, num_games, multiplayer)
        if verbose:
            print(f"   Match ID: {match_id}")

    if verbose:
        print(f"🔗 Joining match {match_id}...")
    match = client.join_match(match_id)
    player = match['player']
    num_games = match.get('num-games', num_games)
    if verbose:
        print(f"   You are player: {player}")

    game_state = client.get_game_state(match_id, 0)
    if game_state['status'] == 'waiting':
        if verbose:
            print("⏳ Waiting for opponent to join...")
        while game_state['status'] == 'waiting':
            time.sleep(2)
            game_state = client.get_game_state(match_id, 0)

    all_results = []
    wins = 0
    losses = 0
    draws = 0

    while True:
        match_state = client.get_match_state(match_id)
        if match_state['status'] != 'in_progress':
            break
        game_num = match_state['current-game-index']

        if verbose:
            print(f"\n{'='*50}")
            print(f"🎮 GAME {game_num + 1}/{num_games}")
            print(f"{'='*50}\n")

        # Get initial game state and check player assignment
        game_state = client.get_game_state(match_id, game_num)
        
        # Update player sign if it changed (randomized per game)
        if 'my-player' in game_state and game_state['my-player']:
            new_player = game_state['my-player']
            if new_player != player and verbose and game_num > 0:
                print(f"ℹ️  Player assignment changed: You are now Player {new_player}\n")
            player = new_player
        
        game = game_class(game_state['state'], game_state['status'], game_state['player'], player)

        game_turn_times: List[float] = []
        move_count = 0
        while game_state['status'] != 'complete':
            game_state = client.get_game_state(match_id, game_num)
            player = game_state['my-player']
            if 'winner' in game_state:
                break

            game = game_class(game_state['state'], game_state['status'], game_state['player'], player)
            
            if game.is_terminal():
                break

            if verbose:
                game.print_board()

            if game.current_player == player:
                if verbose:
                    print(f"🤔 Your turn (Player {player})...")

                try:
                    start_time = time.perf_counter()
                    move = solver(game)
                    elapsed_ms = (time.perf_counter() - start_time) * 1000
                    game_turn_times.append(elapsed_ms)
                    session_turn_times.append(elapsed_ms)

                    if verbose:
                        if hasattr(move, '__iter__') and not isinstance(move, str):
                            if len(move) > 0 and move[0] == 'M':
                                print(f"   Moving pawn to ({move[1]}, {move[2]})")
                            elif len(move) > 0 and move[0] == 'W':
                                print(f"   Placing {move[3]} wall at ({move[1]}, {move[2]})")
                            else:
                                print(f"   Move: {move}")
                        else:
                            print(f"   Move: {move}")
                        print(f"⏱️ Solver time: {elapsed_ms:.1f} ms")

                    client.make_move(match_id, player, move)
                    move_count += 1

                except Exception as e:
                    print(f"❌ Error in solver: {e}")
                    import traceback
                    traceback.print_exc()
                    all_results.append(('error', None))
                    break
            else:
                if verbose:
                    print(f"⏳ Waiting for opponent (Player {game.current_player})...")
                time.sleep(2)

        # game is already terminal from the loop above
        if verbose:
            game.print_board()
            print("=" * 40)
            if game_turn_times:
                avg_ms = sum(game_turn_times) / len(game_turn_times)
                print(f"⏱️ Turn timing this game: avg {avg_ms:.1f} ms, max {max(game_turn_times):.1f} ms over {len(game_turn_times)} turns")

        winner = game_state['winner']
        if winner == '-':
            if verbose:
                print("🤝 Game ended in a DRAW!")
            result = 'draw'
            draws += 1
        elif winner == player:
            if verbose:
                print("🎉 You WON! Congratulations!")
            result = 'win'
            wins += 1
        else:
            if verbose:
                print("😞 You LOST. Better luck next time!")
            result = 'loss'
            losses += 1

        all_results.append((result, player, winner))

        if verbose and num_games > 1:
            print(f"\n📊 Current Record: {wins}W - {losses}L - {draws}D")
            print(f"   Games Remaining: {num_games - game_num - 1}\n")

    if session_turn_times and verbose:
        avg_ms = sum(session_turn_times) / len(session_turn_times)
        print(f"⏱️ Overall solver timing: avg {avg_ms:.1f} ms, max {max(session_turn_times):.1f} ms across {len(session_turn_times)} turns")

    # Return results
    stats = {
        'wins': wins,
        'losses': losses,
        'draws': draws,
        'total_games': num_games,
        'win_rate': wins / num_games if num_games > 0 else 0,
        'player': player,
        'match_id': match_id
    }
    if session_turn_times:
        stats['avg_turn_ms'] = sum(session_turn_times) / len(session_turn_times)
        stats['max_turn_ms'] = max(session_turn_times)

    return stats, all_results

print("✅ play_game loaded")

---
## Section 3: Game State Class & Helper Functions

**Run this cell (no changes needed)**

This defines the `QuoridorGame` class with all helper methods you'll need.

In [ ]:
class QuoridorGame:
    """
    Represents Quoridor game state with helper methods.
    
    Key methods for your solver:
    - game.get_my_position()           # Your pawn position [row, col]
    - game.get_opponent_position()     # Opponent pawn position [row, col]
    - game.get_walls()                 # List of walls [[r, c, 'H'/'V'], ...]
    - game.get_remaining_walls()       # Your remaining wall count
    - game.get_valid_moves()           # All valid moves
    - game.get_valid_pawn_moves()      # Only pawn moves
    - game.get_valid_wall_moves()      # Only wall placements
    - game.simulate_move(move)         # Simulate move for search
    - game.is_terminal()               # Check if game over
    - game.print_board()               # Debug visualization
    """

    def __init__(self, state: str, status: str, current_player: str, my_player: str):
        self.state_str = state
        self.status = status
        self.current_player = current_player
        self.my_player = my_player
        self._state = None
        self._valid_moves = None

    @property
    def state(self) -> Dict:
        """Get state dictionary."""
        if self._state is None:
            self._state = json.loads(self.state_str)
        return self._state

    def get_my_position(self) -> List[int]:
        """Get your pawn position [row, col]."""
        return self.state['pawns'][self.my_player]

    def get_opponent_position(self) -> List[int]:
        """Get opponent pawn position [row, col]."""
        opp = '2' if self.my_player == '1' else '1'
        return self.state['pawns'][opp]

    def get_walls(self) -> List[List]:
        """Get list of walls [[row, col, 'H'/'V'], ...]."""
        return self.state['walls']

    def get_remaining_walls(self, player: Optional[str] = None) -> int:
        """Get remaining wall count for player (defaults to you)."""
        if player is None:
            player = self.my_player
        return self.state['remaining_walls'][player]

    def is_terminal(self) -> bool:
        """Check if game is over."""
        return self.status == 'complete'

    def is_waiting(self) -> bool:
        """Check if waiting for opponent."""
        return self.status == 'waiting'

    def get_winner(self) -> Optional[str]:
        """Get winner ('1', '2', None if ongoing)."""
        if not self.is_terminal():
            return None
        return self.current_player

    def get_opponent(self, player: str) -> str:
        """Get opponent's identifier."""
        return '2' if player == '1' else '1'

    def _is_wall_blocking(self, walls, from_pos, to_pos) -> bool:
        """Check if a wall blocks movement."""
        r1, c1 = from_pos
        r2, c2 = to_pos

        def has_wall(r, c, orient):
            return [r, c, orient] in walls

        # Moving up
        if r2 < r1 and c1 == c2:
            return has_wall(r2, c1, 'H') or has_wall(r2, c1 - 1, 'H')
        # Moving down
        elif r2 > r1 and c1 == c2:
            return has_wall(r1, c1, 'H') or has_wall(r1, c1 - 1, 'H')
        # Moving left
        elif c2 < c1 and r1 == r2:
            return has_wall(r1, c2, 'V') or has_wall(r1 - 1, c2, 'V')
        # Moving right
        elif c2 > c1 and r1 == r2:
            return has_wall(r1, c1, 'V') or has_wall(r1 - 1, c1, 'V')

        return False
    
    def _build_wall_lookup(self, walls) -> set:
        """Fast lookup set for wall membership checks."""
        return {(w[0], w[1], w[2]) for w in walls}

    def _is_wall_blocking_lookup(self, wall_lookup: set, from_pos, to_pos) -> bool:
        """Set-based variant of wall blocking check used by fast BFS routines."""
        r1, c1 = from_pos
        r2, c2 = to_pos

        def has_wall(r, c, orient):
            return (r, c, orient) in wall_lookup

        if r2 < r1 and c1 == c2:
            return has_wall(r2, c1, 'H') or has_wall(r2, c1 - 1, 'H')
        elif r2 > r1 and c1 == c2:
            return has_wall(r1, c1, 'H') or has_wall(r1, c1 - 1, 'H')
        elif c2 < c1 and r1 == r2:
            return has_wall(r1, c2, 'V') or has_wall(r1 - 1, c2, 'V')
        elif c2 > c1 and r1 == r2:
            return has_wall(r1, c1, 'V') or has_wall(r1 - 1, c1, 'V')

        return False

    def _neighbors_without_walls(self, pos, wall_lookup: set):
        """Yield reachable neighbors from pos given a wall lookup."""
        r, c = pos
        for new_r, new_c in [(r - 1, c), (r + 1, c), (r, c - 1), (r, c + 1)]:
            if 0 <= new_r < 9 and 0 <= new_c < 9:
                if not self._is_wall_blocking_lookup(wall_lookup, (r, c), (new_r, new_c)):
                    yield (new_r, new_c)

    def _path_exists_to_goal(self, player: str, wall_lookup: set) -> bool:
        """Check reachability to the goal row with early-exit BFS."""
        start = tuple(self.state['pawns'][player])
        goal_row = 8 if player == '1' else 0
        queue = deque([start])
        visited = {start}

        while queue:
            pos = queue.popleft()
            if pos[0] == goal_row:
                return True
            for neighbor in self._neighbors_without_walls(pos, wall_lookup):
                if neighbor not in visited:
                    visited.add(neighbor)
                    queue.append(neighbor)

        return False

    def _shortest_path_cells(self, player: str, wall_lookup: set) -> List[tuple]:
        """Return one shortest path as a list of cells for heuristic targeting."""
        start = tuple(self.state['pawns'][player])
        goal_row = 8 if player == '1' else 0

        parents = {start: None}
        queue = deque([start])

        while queue:
            pos = queue.popleft()
            if pos[0] == goal_row:
                path = []
                curr = pos
                while curr is not None:
                    path.append(curr)
                    curr = parents[curr]
                return list(reversed(path))

            for neighbor in self._neighbors_without_walls(pos, wall_lookup):
                if neighbor not in parents:
                    parents[neighbor] = pos
                    queue.append(neighbor)

        return []

    def _wall_anchor_distance(self, wall, cell: List[int]) -> int:
        """Manhattan distance from a wall's anchor to a board cell."""
        r, c, _ = wall
        return abs(r - cell[0]) + abs(c - cell[1])

    def _wall_near_point(self, wall, cell: List[int], radius: Optional[int]) -> bool:
        """Check if wall anchor lies within radius of a given cell."""
        if radius is None:
            return True
        return self._wall_anchor_distance(wall, cell) <= radius

    def _wall_near_path(self, wall, path_cells: List[tuple], radius: Optional[int]) -> bool:
        """Check if wall is near any cell on a path."""
        if radius is None:
            return True
        if not path_cells:
            return False
        return any(self._wall_anchor_distance(wall, cell) <= radius for cell in path_cells)

    def _wall_extends_existing(self, wall, wall_lookup: set) -> bool:
        """Light heuristic: prefer walls that extend an existing barrier."""
        r, c, orient = wall
        if orient == 'H':
            return (r, c - 1, 'H') in wall_lookup or (r, c + 1, 'H') in wall_lookup
        return (r - 1, c, 'V') in wall_lookup or (r + 1, c, 'V') in wall_lookup

    def _all_wall_positions(self):
        """Iterate over every legal wall coordinate (without validation)."""
        for r in range(9):
            for c in range(8):
                yield (r, c, 'H')
        for r in range(8):
            for c in range(9):
                yield (r, c, 'V')

    def _generate_wall_candidates(self, player: str, strategy: str, wall_lookup: set,
                                  limit: Optional[int]) -> List[List]:
        """
        Produce a pruned, ordered list of wall candidates using heuristics.

        Strategies:
            - 'path_focus' (default): favor walls near opponent path/pawn and existing barriers.
            - 'proximity': larger net; near either pawn or either shortest path.
            - 'exhaustive': consider every slot (still runs path checks later).
        """
        configs = {
            'path_focus': {'path_radius': 3, 'pawn_radius': 3, 'candidate_cap': 70},
            'proximity': {'path_radius': 4, 'pawn_radius': 4, 'candidate_cap': 110},
            'exhaustive': {'path_radius': None, 'pawn_radius': None, 'candidate_cap': None}
        }
        cfg = configs.get(strategy, configs['path_focus'])

        # Inflate candidate cap a bit when a limit is requested so we still find enough valid walls.
        candidate_cap = cfg['candidate_cap']
        if limit is not None:
            if candidate_cap is None:
                candidate_cap = limit * 3
            else:
                candidate_cap = max(candidate_cap, limit * 2)

        opponent = self.get_opponent(player)
        my_pos = self.state['pawns'][player]
        opp_pos = self.state['pawns'][opponent]

        opp_path = self._shortest_path_cells(opponent, wall_lookup)
        my_path = self._shortest_path_cells(player, wall_lookup)

        scored_candidates = []
        for wall in self._all_wall_positions():
            extends = self._wall_extends_existing(wall, wall_lookup)
            near_opp_path = self._wall_near_path(wall, opp_path, cfg['path_radius'])
            near_my_path = self._wall_near_path(wall, my_path, cfg['path_radius'])
            near_opp_pawn = self._wall_near_point(wall, opp_pos, cfg['pawn_radius'])
            near_my_pawn = self._wall_near_point(wall, my_pos, cfg['pawn_radius'])

            if strategy == 'exhaustive':
                keep = True
            elif strategy == 'proximity':
                keep = near_opp_path or near_my_path or near_opp_pawn or near_my_pawn or extends
            else:  # path_focus
                keep = near_opp_path or (near_opp_pawn and (near_opp_path or near_my_path)) or (extends and (near_opp_pawn or near_opp_path))

            if not keep:
                continue

            score = 0.0
            if near_opp_path:
                score += 3.0
            if near_opp_pawn:
                score += 1.8
            if extends:
                score += 0.8
            if near_my_pawn:
                score += 0.5
            if near_my_path:
                score += 0.2

            score -= self._wall_anchor_distance(wall, opp_pos) * 0.05
            scored_candidates.append((score, [wall[0], wall[1], wall[2]]))

        scored_candidates.sort(key=lambda x: x[0], reverse=True)
        if candidate_cap is not None:
            scored_candidates = scored_candidates[:candidate_cap]

        return [wall for _, wall in scored_candidates]

    def _wall_preserves_paths(self, wall, player: str, wall_lookup: set) -> bool:
        """Check that adding wall keeps at least one path for both players."""
        new_lookup = set(wall_lookup)
        new_lookup.add((wall[0], wall[1], wall[2]))
        opponent = self.get_opponent(player)

        if not self._path_exists_to_goal(opponent, new_lookup):
            return False
        return self._path_exists_to_goal(player, new_lookup)

    def get_valid_pawn_moves(self, player: Optional[str] = None) -> List[List]:
        """Get valid pawn moves for player."""
        if player is None:
            player = self.current_player

        state = self.state
        my_pos = state['pawns'][player]
        opp_pos = state['pawns'][self.get_opponent(player)]
        walls = state['walls']

        r, c = my_pos
        moves = []

        # Basic adjacent moves
        adjacent = [(r - 1, c), (r + 1, c), (r, c - 1), (r, c + 1)]

        for new_r, new_c in adjacent:
            if 0 <= new_r < 9 and 0 <= new_c < 9:
                if [new_r, new_c] == opp_pos:
                    # Try to jump over opponent
                    dr = new_r - r
                    dc = new_c - c
                    jump_r = new_r + dr
                    jump_c = new_c + dc

                    if 0 <= jump_r < 9 and 0 <= jump_c < 9:
                        if not self._is_wall_blocking(walls, opp_pos, [jump_r, jump_c]):
                            moves.append(['M', jump_r, jump_c])
                    else:
                        # Diagonal jumps when can't jump straight
                        if dr != 0:  # Moving vertically, try left/right
                            for side_dc in [-1, 1]:
                                side_c = new_c + side_dc
                                if 0 <= side_c < 9:
                                    if not self._is_wall_blocking(walls, opp_pos, [new_r, side_c]):
                                        moves.append(['M', new_r, side_c])
                        else:  # Moving horizontally, try up/down
                            for side_dr in [-1, 1]:
                                side_r = new_r + side_dr
                                if 0 <= side_r < 9:
                                    if not self._is_wall_blocking(walls, opp_pos, [side_r, new_c]):
                                        moves.append(['M', side_r, new_c])
                else:
                    # Normal move
                    if not self._is_wall_blocking(walls, my_pos, [new_r, new_c]):
                        moves.append(['M', new_r, new_c])

        return moves

    def get_valid_wall_moves(self, player: Optional[str] = None, limit: int = None,
                             strategy: str = "path_focus") -> List[List]:
        """
        Get valid wall placements for player using heuristics and path safety checks.
        
        Args:
            player: Player to get walls for (defaults to current player)
            limit: Maximum number of wall moves to return after validation
            strategy: Wall generation strategy ('path_focus', 'proximity', 'exhaustive')
        """
        if player is None:
            player = self.current_player

        if self.get_remaining_walls(player) == 0:
            return []

        walls = self.get_walls()
        wall_lookup = self._build_wall_lookup(walls)
        candidates = self._generate_wall_candidates(player, strategy, wall_lookup, limit)

        moves = []
        for wall in candidates:
            if not self._is_valid_wall_simple(wall, walls):
                continue
            if not self._wall_preserves_paths(wall, player, wall_lookup):
                continue
            moves.append(['W', wall[0], wall[1], wall[2]])
            if limit is not None and len(moves) >= limit:
                break

        # If a tight heuristic returns nothing, retry with exhaustive once.
        if not moves and strategy != 'exhaustive':
            return self.get_valid_wall_moves(player=player, limit=limit, strategy='exhaustive')

        return moves

    def _is_valid_wall_simple(self, wall, existing_walls) -> bool:
        """Simplified wall validation (doesn't check pathfinding)."""
        r, c, orient = wall

        # Check if wall already exists
        if wall in existing_walls:
            return False

        # Check for overlaps and crossings (simplified)
        if orient == 'H':
            if [r, c + 1, 'H'] in existing_walls or [r, c - 1, 'H'] in existing_walls:
                return False
            if [r - 1, c, 'V'] in existing_walls or [r, c, 'V'] in existing_walls:
                return False
            if [r - 1, c + 1, 'V'] in existing_walls or [r, c + 1, 'V'] in existing_walls:
                return False
        else:  # 'V'
            if [r + 1, c, 'V'] in existing_walls or [r - 1, c, 'V'] in existing_walls:
                return False
            if [r, c - 1, 'H'] in existing_walls or [r, c, 'H'] in existing_walls:
                return False
            if [r + 1, c - 1, 'H'] in existing_walls or [r + 1, c, 'H'] in existing_walls:
                return False

        # NOTE: This doesn't check if wall blocks all paths!
        # The server will reject invalid walls, but checking here is expensive
        return True

    def get_valid_moves(self, player: Optional[str] = None, limit_walls: int = 20) -> List[List]:
        """
        Get all valid moves.
        
        Args:
            player: Player to get moves for
            limit_walls: Limit wall checks for performance (set to None for all)
        """
        if self._valid_moves is None or player != self.current_player:
            pawn_moves = self.get_valid_pawn_moves(player)
            wall_moves = self.get_valid_wall_moves(player, limit=limit_walls)
            self._valid_moves = pawn_moves + wall_moves
        return self._valid_moves

    def simulate_move(self, move: List) -> Dict:
        """
        Simulate a move and return new state.
        Does NOT contact server or modify original state.
        Essential for minimax/alpha-beta, but this can be expensive because of deepcopy.
        """
        new_state = deepcopy(self.state)
        player = self.current_player

        if move[0] == 'M':
            # Pawn move
            new_state['pawns'][player] = [move[1], move[2]]
        elif move[0] == 'W':
            # Wall placement
            new_state['walls'].append([move[1], move[2], move[3]])
            new_state['remaining_walls'][player] -= 1

        return new_state

    def shortest_path_length(self, player: Optional[str] = None, state: Optional[Dict] = None) -> int:
        """
        Calculate shortest path length to goal using BFS.
        Useful for evaluation functions!
        
        Returns:
            int: Number of moves to goal, or 999 if no path
        """
        if player is None:
            player = self.my_player
        if state is None:
            state = self.state

        start = tuple(state['pawns'][player])
        goal_row = 8 if player == '1' else 0
        walls = state['walls']

        visited = {start: 0}
        queue = deque([start])

        while queue:
            r, c = queue.popleft()
            dist = visited[(r, c)]

            if r == goal_row:
                return dist

            for new_r, new_c in [(r-1, c), (r+1, c), (r, c-1), (r, c+1)]:
                if 0 <= new_r < 9 and 0 <= new_c < 9:
                    if (new_r, new_c) not in visited:
                        if not self._is_wall_blocking(walls, [r, c], [new_r, new_c]):
                            visited[(new_r, new_c)] = dist + 1
                            queue.append((new_r, new_c))

        return 999  # No path found

    def print_board(self):
        """Print nice board visualization."""
        state = self.state
        pawns = state['pawns']
        walls = state['walls']

        print("\n" + "=" * 37)
        print(f"Player 1: {pawns['1']}  Player 2: {pawns['2']}")
        print(f"Walls remaining - You: {self.get_remaining_walls(self.my_player)}, " +
              f"Opponent: {self.get_remaining_walls(self.get_opponent(self.my_player))}")
        print("=" * 37)

        # Create visual board
        for r in range(9):
            # Print row with cells
            row_str = ""
            for c in range(9):
                if pawns['1'] == [r, c]:
                    row_str += "1"
                elif pawns['2'] == [r, c]:
                    row_str += "2"
                else:
                    row_str += "·"

                # Check for vertical wall to the right
                if c < 8:
                    if [r, c, 'V'] in walls or (r > 0 and [r - 1, c, 'V'] in walls):
                        row_str += " ║ "
                    else:
                        row_str += "   "

            print(row_str)

            # Print horizontal walls
            if r < 8:
                wall_str = ""
                for c in range(9):
                    if [r, c, 'H'] in walls or (c > 0 and [r, c - 1, 'H'] in walls):
                        wall_str += "═"
                    else:
                        wall_str += " "

                    if c < 8:
                        wall_str += "   "

                print(wall_str)

        print("=" * 37 + "\n")

        """Print nice board visualization."""
        state = self.state
        pawns = state['pawns']
        walls = state['walls']

        print("\n" + "=" * 37)
        print(f"Player 1: {pawns['1']}  Player 2: {pawns['2']}")
        print(f"Walls remaining - You: {self.get_remaining_walls(self.my_player)}, " +
              f"Opponent: {self.get_remaining_walls(self.get_opponent(self.my_player))}")
        print("=" * 37)

        # Create visual board
        for r in range(9):
            # Print row with cells
            row_str = ""
            for c in range(9):
                if pawns['1'] == [r, c]:
                    row_str += "1"
                elif pawns['2'] == [r, c]:
                    row_str += "2"
                else:
                    row_str += "·"

                # Check for vertical wall to the right
                if c < 8:
                    if [r, c, 'V'] in walls or (r > 0 and [r - 1, c, 'V'] in walls):
                        row_str += " ║ "
                    else:
                        row_str += "   "

            print(row_str)

            # Print horizontal walls
            if r < 8:
                wall_str = ""
                for c in range(9):
                    if [r, c, 'H'] in walls or (c > 0 and [r, c - 1, 'H'] in walls):
                        wall_str += "═"
                    else:
                        wall_str += " "

                    if c < 8:
                        wall_str += "   "

                print(wall_str)

        print("=" * 37 + "\n")

---
## Section 4: Manual Play Mode (Try the Game Yourself!)

**Play Quoridor manually** to understand the game before implementing your AI!

This lets you:
- Experience the game firsthand
- Test the server connection
- Understand winning strategies
- Play against the server AI

In [ ]:
def manual_player_solver(game: QuoridorGame) -> List:
    """
    Interactive manual player - YOU choose the moves!
    Perfect for testing the game and understanding the rules.
    """
    game.print_board()
    
    pawn_moves = game.get_valid_pawn_moves()
    wall_moves = game.get_valid_wall_moves(limit=30)
    
    print(f"\n🎮 YOUR TURN (Player {game.my_player})!")
    print("\nValid pawn moves:")
    for i, move in enumerate(pawn_moves):
        print(f"  {i}: Move to ({move[1]}, {move[2]})")
    
    if wall_moves and game.get_remaining_walls() > 0:
        print(f"\nOr place a wall (you have {game.get_remaining_walls()} left):")
        print("  Enter: W row col H/V (e.g., 'W 3 4 H')")
    
    while True:
        try:
            choice = input("\nEnter your choice (number for pawn move, or 'W r c H/V' for wall): ").strip()
            
            if choice.lower() == 'q':
                raise KeyboardInterrupt()
            
            if choice.upper().startswith('W'):
                parts = choice.split()
                if len(parts) == 4:
                    move = ['W', int(parts[1]), int(parts[2]), parts[3].upper()]
                    return move
                else:
                    print("❌ Invalid format! Use: W row col H/V")
            else:
                idx = int(choice)
                if 0 <= idx < len(pawn_moves):
                    return pawn_moves[idx]
                else:
                    print(f"❌ Invalid index! Choose 0-{len(pawn_moves)-1}")
        
        except ValueError:
            print("❌ Invalid input! Enter a number or 'W row col H/V'")
        except KeyboardInterrupt:
            print("\n👋 Thanks for playing!")
            raise

print("✅ Manual player loaded")
print("   Run the cell below to play interactively!")

---
## Section 5: YOUR SOLVER IMPLEMENTATION

**⭐ THIS IS WHERE YOU WRITE YOUR CODE! ⭐**

Implement your AI algorithm here. You can use:
- Minimax
- Alpha-beta pruning
- Custom heuristics
- A* pathfinding

### Available Methods

```python
game.get_my_position()                    # Your pawn [row, col]
game.get_opponent_position()              # Opponent pawn [row, col]
game.get_walls()                          # List of walls
game.get_remaining_walls(player)          # Wall count
game.get_valid_moves()                    # All valid moves
game.get_valid_pawn_moves()               # Only pawn moves
game.get_valid_wall_moves(limit=20)       # Only walls (limited for speed)
game.simulate_move(move)                  # Simulate move, returns new state
game.shortest_path_length(player, state)  # BFS path length to goal
game.print_board()                        # Print board
```

### Move Format
- Pawn move: `['M', row, col]`
- Wall placement: `['W', row, col, 'H']` or `['W', row, col, 'V']`

### Strategy Tips
1. Use `shortest_path_length()` in your evaluation function
2. Balance pawn advancement vs. blocking opponent with walls
3. Be careful with wall placement - checking all positions is slow!
4. Consider limiting search depth due to high branching factor

In [ ]:
def my_agent(game: QuoridorGame) -> List:
    """
    Your AI implementation.
    
    Args:
        game: QuoridorGame object with helper methods
    
    Returns:
        List: Move in format ['M', row, col] or ['W', row, col, 'H'/'V']
    """
    
    # Basic identifiers
    my_player = game.my_player
    opponent = game.get_opponent(my_player)
    root_state = game.state

    # Search depth: keep shallow when walls are abundant, look a bit deeper in simplified endgames
    search_depth = 2
    total_walls = root_state['remaining_walls']['1'] + root_state['remaining_walls']['2']
    if total_walls <= 4:
        search_depth = 3

    def opponent_of(player: str) -> str:
        return '2' if player == '1' else '1'

    def apply_move(state: Dict, player: str, move: List) -> Dict:
        """Lightweight move application on a state dictionary."""
        new_state = deepcopy(state)
        if move[0] == 'M':
            new_state['pawns'][player] = [move[1], move[2]]
        else:
            new_state['walls'].append([move[1], move[2], move[3]])
            new_state['remaining_walls'][player] -= 1
        return new_state

    def evaluate_state(state: Dict) -> float:
        """Heuristic evaluation using shortest path lengths and wall advantage."""
        my_dist = game.shortest_path_length(my_player, state)
        opp_dist = game.shortest_path_length(opponent, state)

        if my_dist == 0:
            return 1e6
        if opp_dist == 0:
            return -1e6

        wall_diff = state['remaining_walls'][my_player] - state['remaining_walls'][opponent]
        wall_penalty = -2 if state['remaining_walls'][my_player] == 0 and state['remaining_walls'][opponent] > 0 else 0

        # Favor being ahead in the race (distance difference), keep some value on spare walls
        return (opp_dist - my_dist) * 12 + wall_diff * 1.5 + wall_penalty

    def should_sprint(state: Dict, player: str) -> bool:
        """
        Decide if we should ignore walls and just run (e.g., opponent has no walls,
        or we are ahead enough with few walls left).
        """
        my_walls = state['remaining_walls'][player]
        opp_walls = state['remaining_walls'][opponent_of(player)]
        if my_walls == 0 and opp_walls > 0:
            return True
        if opp_walls == 0:
            return True

        my_dist = game.shortest_path_length(player, state)
        opp_dist = game.shortest_path_length(opponent_of(player), state)

        if my_dist + 1 < opp_dist and opp_walls <= 1:
            return True
        return False

    def candidate_wall_moves(state: Dict, player: str) -> List[Tuple[float, List]]:
        """
        Score wall moves by how much they hurt the opponent without over-harming us.
        Returns a list of (score, move) pairs.
        """
        if state['remaining_walls'][player] == 0:
            return []

        temp_game = QuoridorGame(json.dumps(state), 'playing', player, player)
        raw_walls = temp_game.get_valid_wall_moves(player, limit=None)
        if not raw_walls:
            return []

        opp = opponent_of(player)
        my_pos = state['pawns'][player]
        opp_pos = state['pawns'][opp]
        existing = {(w[0], w[1], w[2]) for w in state['walls']}
        base_my = game.shortest_path_length(player, state)
        base_opp = game.shortest_path_length(opp, state)

        scored = []
        for move in raw_walls:
            _, r, c, orient = move
            near_opp = abs(r - opp_pos[0]) + abs(c - opp_pos[1]) <= 3
            near_me = abs(r - my_pos[0]) + abs(c - my_pos[1]) <= 3

            # Encourage extending existing barriers
            extends = False
            if orient == 'H':
                extends = ((r, c - 1, 'H') in existing or (r, c + 1, 'H') in existing)
            else:
                extends = ((r - 1, c, 'V') in existing or (r + 1, c, 'V') in existing)

            # Opening guideline: avoid random walls very early
            if len(state['walls']) <= 1 and state['remaining_walls'][player] >= 8:
                if not (near_opp or extends):
                    continue

            if not (near_opp or near_me or extends):
                continue

            next_state = apply_move(state, player, move)
            new_opp = game.shortest_path_length(opp, next_state)
            new_my = game.shortest_path_length(player, next_state)

            opp_gain = new_opp - base_opp
            my_cost = max(0, new_my - base_my)

            # Demand the wall to add real tax to the opponent and not hurt us too much
            if opp_gain < 1:
                continue

            score = opp_gain - 0.8 * my_cost
            if near_opp:
                score += 0.4
            if extends:
                score += 0.25

            # If we are already well ahead, only keep walls that add a big detour
            if (base_opp - base_my) >= 2 and opp_gain < 2:
                continue

            scored.append((score, move))

        scored.sort(key=lambda x: x[0], reverse=True)
        return scored[:8]  # keep top candidates to control branching

    def generate_moves(state: Dict, player: str) -> List[List]:
        """Generate ordered moves for alpha-beta search."""
        temp_game = QuoridorGame(json.dumps(state), 'playing', player, player)
        pawn_moves = temp_game.get_valid_pawn_moves(player)

        # Order pawn moves by how much they shorten our path
        base_dist = game.shortest_path_length(player, state)
        pawn_scored = []
        for mv in pawn_moves:
            next_state = apply_move(state, player, mv)
            new_dist = game.shortest_path_length(player, next_state)
            pawn_scored.append((base_dist - new_dist, mv))
        pawn_scored.sort(key=lambda x: x[0], reverse=True)

        # Toggle race mode when walls are ineffective
        if should_sprint(state, player):
            return [mv for _, mv in pawn_scored]

        wall_scored = candidate_wall_moves(state, player)
        ordered = [mv for _, mv in pawn_scored] + [mv for _, mv in wall_scored]
        return ordered

    def minimax(state: Dict, player: str, depth: int, alpha: float, beta: float) -> float:
        my_turn = player == my_player
        my_dist = game.shortest_path_length(my_player, state)
        opp_dist = game.shortest_path_length(opponent, state)

        # Quick terminal checks
        if my_dist == 0:
            return 1e6
        if opp_dist == 0:
            return -1e6
        if depth == 0:
            return evaluate_state(state)

        moves = generate_moves(state, player)
        if not moves:
            return evaluate_state(state)

        if my_turn:
            value = -float('inf')
            for mv in moves:
                child_state = apply_move(state, player, mv)
                value = max(value, minimax(child_state, opponent_of(player), depth - 1, alpha, beta))
                alpha = max(alpha, value)
                if beta <= alpha:
                    break
            return value
        else:
            value = float('inf')
            for mv in moves:
                child_state = apply_move(state, player, mv)
                value = min(value, minimax(child_state, opponent_of(player), depth - 1, alpha, beta))
                beta = min(beta, value)
                if beta <= alpha:
                    break
            return value

    # Root search
    best_score = -float('inf')
    best_move = None
    root_moves = generate_moves(root_state, my_player)

    # If we somehow have no generated moves, fall back to any legal move
    if not root_moves:
        fallback = game.get_valid_pawn_moves() + game.get_valid_wall_moves(limit=10)
        return random.choice(fallback)

    for move in root_moves:
        next_state = apply_move(root_state, my_player, move)
        score = minimax(next_state, opponent, search_depth - 1, -float('inf'), float('inf'))
        if score > best_score:
            best_score = score
            best_move = move

    return best_move if best_move is not None else random.choice(root_moves)


---
## Section 7: Play the Game!

**Run this cell to test your solver against the AI**

In [ ]:
STUDENT_TOKEN = 'BORIS-GANS'  # e.g., 'JOHN-DOE'
SOLVER = my_agent
MULTIPLAYER = False
MATCH_ID = None
NUM_GAMES = 20

# Wall generation tuning: options are 'path_focus', 'proximity', 'exhaustive'
# WALL_STRATEGY = "path_focus"

result = play_game(
    solver=SOLVER,
    base_url=BASE_URL,
    token=STUDENT_TOKEN,
    game_type='quoridor',
    game_class=QuoridorGame,
    multiplayer=MULTIPLAYER,
    match_id=MATCH_ID,
    num_games=NUM_GAMES,
    debug=False,
    verbose=True
)

stats, all_results = result
print("\n📊 Summary:")
print(f"   Record: {stats['wins']}W - {stats['losses']}L - {stats['draws']}D")
print(f"   Win Rate: {stats['win_rate']*100:.1f}%")